# Fetch Media List
* Queries a user's anime and manga lists

In [3]:
import glob
import logging
import os
import time
from io import StringIO

import pandas as pd
from tqdm import tqdm

In [ ]:
username = ""
source = ""

In [ ]:
logging.getLogger().setLevel(logging.INFO)

In [ ]:
PROXY_NUMBER = 0
TOKEN_NUMBER = 0

In [ ]:
allowed_sources = ["mal", "anilist", "kitsu", "animeplanet", "training"]
assert source in allowed_sources

In [ ]:
logger = logging.getLogger("FetchMediaLists")
logger.setLevel(logging.DEBUG)
formatter = logging.Formatter(
    "%(name)s:%(levelname)s:%(asctime)s: %(message)s", datefmt="%Y-%m-%d %H:%M:%S"
)
for stream in [
    logging.StreamHandler(),
]:
    stream.setFormatter(formatter)
    logger.addHandler(stream)

In [ ]:
logging.info(f"Fetching lists for {username} from {source}")

In [ ]:
def import_from_api(username, medium, source):
    pwd = os.getcwd()
    try:
        os.chdir("../API/API")
        if source == "mal":
            %run MalApi.ipynb
        elif source == "anilist":
            %run AnilistApi.ipynb
            if isinstance(username, str):
                username = get_userid(username)
        elif source == "kitsu":
            if isinstance(username, str):
                username = get_userid(username)
            %run KitsuApi.ipynb
        elif source == "animeplanet":
            %run AnimeplanetApi.ipynb
        df, ret = get_user_media_list(username, medium)
        if not ret:
            raise Exception(f"Could not resolve list for {username}")
    finally:
        os.chdir(pwd)
    return df

In [ ]:
def import_from_training(username, medium):
    s, usertag = username.split("@")
    lines = []

    files = glob.glob(f"../../data/{s}/user_media_facts/user_status.*.csv")
    fn = None
    for f in tqdm(files):
        df = pd.read_csv(f).query(f"username == '{usertag}'")
        if not df.empty:
            fn = f
            break
    assert fn is not None, f"could not find {username}"
    media_fn = f.replace("user_status", f"user_{medium}_list")
    df = pd.read_csv(media_fn).query(f"username == '{usertag}'")
    return df

In [ ]:
def import_list(userid, medium, source):
    if source == "training":
        return import_from_training(username, medium)
    else:
        return import_from_api(username, medium, source)

In [1]:
def save_path(source, username, medium):
    if source == "training":
        source, username = username.split("@")
    data_path = os.path.join("../../data/recommendations", source, str(username))
    if not os.path.exists(data_path):
        os.makedirs(data_path)
    return os.path.join(data_path, f"user_{medium}_list.{source}.csv")

In [ ]:
for medium in ["manga", "anime"]:
    df = import_list(username, medium, source)
    df.to_csv(save_path(source, username, medium), index=False)